In [1]:
import numpy as np
import pandas as pd
from scipy.stats import gmean
from glob import glob
import math
from bs4 import BeautifulSoup
import requests
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
Annual = "Y"
Quarterly = "Q"

# FnGuide 크롤링, 분기 영활현이 없을시 NaN value
def crawlCFdata(code, frequency):
    # Using FnGuide
    URL = f"https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode={code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701"
    response = requests.get(URL)
    soup = BeautifulSoup(response.content,'html.parser')
    
    # Getting CF figures
    divCash = soup.find("div",{"id":f"divCash{frequency}"})
    table = divCash.find("table",{"class":"us_table_ty1 h_fix zigbg_no"})
    tbody = table.find("tbody")
    trs = tbody.findAll("tr")
    CFtr = trs[0]
    CFtd = CFtr.findAll("td")
    
    # some companies does not have data
    CFlist = []
    for dpoint in range(len(CFtd)):
        dpoint = CFtd[dpoint].text.replace(",","").replace(u'\xa0', u'nan')
        if dpoint == 'nan':
            CFlist.append(dpoint)
        else:
            CFlist.append(float(dpoint))

    #Getting Dates of CF figures
    Dateth = table.findAll("th")[1:5]

    Dates = []
    for date in Dateth:
        Dates.append(date.text)

    return Dates, CFlist


# 크롤링 데이터 frequency input & outputs pd.Series of OCF for each firms
def getCF(code, frequency):
    #for code in code_list:
    if frequency == "Q":
        # Form Quarterly Operating CF series
        Qcf = crawlCFdata(code, Quarterly)
        Qcf_ser = pd.Series(Qcf[1],index = Qcf[0],name = f"{code}")
        return Qcf_ser
        
    elif frequency == "Y":
        # Form Annual Operating CF series
        Ycf = crawlCFdata(code, Annual)
        Ycf_ser = pd.Series(Ycf[1],index = Ycf[0],name = f"{code}")
        return Ycf_ser


# Input exact Year & Quarter to get OCF specific to the date for each firm
def searchCF(code,frequency,Year,Month=None):
    companyCF = getCF(code, frequency)

    for time in companyCF.index:
        if frequency == Quarterly:
            if Year in time:
                if Month in time:
                    return float(companyCF.loc[time])

        elif frequency == Annual:
            if Year in time:
                return float(companyCF.loc[time])            


# 파일명을 사용해서 자동 해당 분기 네이밍
def naming(path):
    download_date = path.split("_")[1]
    YY = int(download_date[2:4])
    MM = download_date[4:6]
    Q = Qclassify[MM]
    if Q == '4Q':
        YY -= 1
    return YY,Q

# 분기 백워드 카운터
def backwardQcounter(YY,Q,N): # N = number of backward counts
    Qlist = ['4Q','3Q','2Q','1Q']*3

    backmove = Qlist.index(Q) + N
    Qstart = Qlist[backmove]
    YYstart = YY - math.ceil((backmove+1)/4) + 1 # 연 단위 백무브 = (이번 분기 포함 총 백무브)/4 + 1(1Q는 같은 연도)
    return YYstart, Qstart

# Settings: a folder with non-redundant Quant Data files, 
# renaming is not necessary as long as it includes ~_20XXMMDD~
files = glob("/Users/kevinpark/Desktop/QuantData/*") #/Users/kevinpark/Desktop/QuantData/*.csv
files

['/Users/kevinpark/Desktop/QuantData/퀀트데이터(전체)_20191215_19Q3.csv',
 '/Users/kevinpark/Desktop/QuantData/퀀트데이터(전체)_20220615_22Q1.csv',
 '/Users/kevinpark/Desktop/QuantData/퀀트데이터(전체)_20220915_164124.csv',
 '/Users/kevinpark/Desktop/QuantData/퀀트데이터(전체)_20190915_19Q2.csv',
 '/Users/kevinpark/Desktop/QuantData/~$퀀트데이터(전체)_20190915_19Q2.xlsx',
 '/Users/kevinpark/Desktop/QuantData/퀀트데이터(전체)_20220415_21Q4.csv',
 '/Users/kevinpark/Desktop/QuantData/퀀트데이터(전체)_20221214_22Q3.csv',
 '/Users/kevinpark/Desktop/QuantData/퀀트데이터(전체)_20221215_194726.csv',
 '/Users/kevinpark/Desktop/QuantData/퀀트데이터(전체)_20210915_21Q2.csv',
 '/Users/kevinpark/Desktop/QuantData/퀀트데이터(전체)_20201215_20Q3.csv',
 '/Users/kevinpark/Desktop/QuantData/과거 분기 15일 이후 근접일',
 '/Users/kevinpark/Desktop/QuantData/퀀트데이터2022.12.14(22년3Q 재무데이터반영).xlsx',
 '/Users/kevinpark/Desktop/QuantData/퀀트데이터(전체)_20200615_20Q1.

In [3]:
### 21Q2(check), 21Q3(check), 21 Q4(check), 22Q1(check)

# Computing Periods settings
# Q4:Y/04, Q1:Y/06, Q2:Y/09, Q3:Y/12 or Q1:Y/06, Q2:Y/09, Q3:Y/12, Q4:Y-1/04
Qclassify = {'06':'1Q','09':'2Q','12':'3Q','04':'4Q'}
YY = 0
Q = 0
YY0 = 0 
Q0 = 0

# 분석 대상 컬럼
identifer = ['코드번호']
valuations = ['회사명','시가총액(억)','발표PSR','발표PBR','발표PER','영업활동현금흐름(억)','EBITDA(억)','과거EV/EBITDA(%)','발표POR']

# 현 분기 퀀트데이터
# INPUT: PATH VARIABLE
path = '/Users/kevinpark/Desktop/QuantData/퀀트데이터(전체)_20221215_194726.csv'
CurrentData = pd.read_csv(path)
CurrentData.columns = CurrentData.columns.str.replace(' ', '') # strip all spaces in column names

CurrentData = CurrentData[CurrentData['업종(소)']!="스팩"]

QuantColumns = list(CurrentData.columns)

# 현 분기 데이터 순이익 추출 = 현 분기 - 7분기
YY,Q = naming(path)
YY0, Q0 = backwardQcounter(YY,Q,N=7)
print(YY,Q,YY0, Q0)

# 컬럼 이름 띄어쓰기 영향 받음
NIstart = QuantColumns.index(f'순이익{YY0}년{Q0}') 
NIend = QuantColumns.index(f'순이익{YY}년{Q}') # 기준: 제일 최근 데이터 = 현 분기
NIcol = QuantColumns[NIstart:NIend+1]
print("현 분기 순이익 데이터:",(YY0, Q0),"부터", (YY, Q))

# 현 분기 데이터 영업이익 추출 = 현 분기 - 7분기
# 컬럼 이름 띄어쓰기 영향 받음
OPstart = QuantColumns.index(f'영업이익{YY0}년{Q0}')
OPend = QuantColumns.index(f'영업이익{YY}년{Q}') # 기준: 제일 최근 데이터 = 현 분기
OPcol = QuantColumns[OPstart:OPend+1]
print("현 분기 영업이익 데이터:", (YY0, Q0),"부터", (YY, Q))

# 현 분기 전체 분석 대상 컬럼 데이터 추출
filteredcols = identifer + valuations + OPcol + NIcol
CurrentData = CurrentData[filteredcols]


# Convert all object dtype to float in the DF
cols = ['시가총액(억)','발표PSR','발표PBR','발표PER','영업활동현금흐름(억)','EBITDA(억)','과거EV/EBITDA(%)','발표POR']
for col in cols:
    if CurrentData.loc[:,col].dtype == 'O':
        CurrentData.loc[:,col] = CurrentData.loc[:,col].str.replace(",","").astype(float)
    else:
        continue

CurrentData.head(2)

22 3Q 20 4Q
현 분기 순이익 데이터: (20, '4Q') 부터 (22, '3Q')
현 분기 영업이익 데이터: (20, '4Q') 부터 (22, '3Q')


,코드번호,회사명,시가총액(억),발표PSR,발표PBR,발표PER,영업활동현금흐름(억),EBITDA(억),과거EV/EBITDA(%),발표POR,...,영업이익22년2Q,영업이익22년3Q,순이익20년4Q,순이익21년1Q,순이익21년2Q,순이익21년3Q,순이익21년4Q,순이익22년1Q,순이익22년2Q,순이익22년3Q
0,A000020,동화약품,2757,0.83,0.77,12.04,445,455,3.28,8.33,...,101,74,78,54,63,4,57,52,60,59
1,A000040,KR모터스,547,0.43,1.85,-5.58,-72,-23,-37.09,-8.55,...,-8,-13,-56,-14,-18,-17,-46,-22,4,-34


In [4]:
# YoY Data Guide
download_date = path.split("_")[1]
YY = int(download_date[2:4])
MM = download_date[4:6]
recommended_date = "20"+str(YY-1)+MM

for file in files:
    if "_" in file:
        if recommended_date in file.split("_")[1]:
            print("Recommended Data:",'\n',file)
            break
    else:
        continue

print(file)

Recommended Data: 
 /Users/kevinpark/Desktop/QuantData/퀀트데이터(전체)_20211214_21Q3.csv
/Users/kevinpark/Desktop/QuantData/퀀트데이터(전체)_20211214_21Q3.csv


In [5]:
# 동일 분기 YoY 퀀트데이터
# INPUT: PATH VARIABLE
path_yoy = file
YOYData = pd.read_csv(path_yoy)
YOYData.columns = YOYData.columns.str.replace(' ', '') # strip all spaces in column names


YOYData = YOYData[YOYData['업종(소)']!="스팩"]

QuantColumns = list(YOYData.columns)

# 현 분기 시작 데이터
YY,Q = naming(path)
YY0, Q0 = backwardQcounter(YY,Q,N=7)

# 동일 분기 YoY 순이익 추출: 현 분기 시작 데이터 - 3분기
N=3
YYstart, Qstart = backwardQcounter(YY0,Q0,N+1) # 현 분기 데이터 마지막 백무브부터 시작, 1번 더 내려가야 됨
YYend, Qend = backwardQcounter(YY0,Q0,N=1)
print("동일 분기 YoY 순이익 시작:",(YYstart, Qstart),"부터", (YYend, Qend))

NIstart = QuantColumns.index(f'순이익{YYstart}년{Qstart}')
NIend = QuantColumns.index(f'순이익{YYend}년{Qend}')
NIyoycol = QuantColumns[NIstart:NIend+1]


# 동일 분기 YoY 영업이익 추출: 현 분기 마지막 데이터 - 2분기
N=2
YYstart, Qstart = backwardQcounter(YY0,Q0,N+1) # 현 분기 데이터 마지막 백무브부터 시작, 1번 더 내려가야 됨
YYend, Qend = backwardQcounter(YY0,Q0,N=1)
print("동일 분기 YoY 영업이익 시작:",(YYstart, Qstart),"부터", (YYend, Qend))

OPstart = QuantColumns.index(f'영업이익{YYstart}년{Qstart}')
OPend = QuantColumns.index(f'영업이익{YYend}년{Qend}')
OPyoycol = QuantColumns[OPstart:OPend+1]


# 동일 분기 YoY 전체 분석 대상 컬럼 데이터 추출
filteredyoycols = identifer + ['회사명'] + OPyoycol + NIyoycol
YOYData = YOYData[filteredyoycols]

YOYData.head(2)

동일 분기 YoY 순이익 시작: (19, '4Q') 부터 (20, '3Q')
동일 분기 YoY 영업이익 시작: (20, '1Q') 부터 (20, '3Q')


,코드번호,회사명,영업이익20년1Q,영업이익20년2Q,영업이익20년3Q,순이익19년4Q,순이익20년1Q,순이익20년2Q,순이익20년3Q
0,A005930,삼성전자,"64,473","81,463","123,532","52,280","48,896","54,890","92,668"
1,A000660,SK하이닉스,"8,028","19,491","13,019","-1,255","6,329","12,689","10,839"


In [6]:
# Merge on 코드 번호
TotalData = pd.merge(YOYData, CurrentData, how='inner', on = identifer[0])
TotalData.set_index(['코드번호'], inplace = True)
#TotalData["회사명_x"] == TotalData["회사명_y"] # all equal
TotalData.drop(["회사명_y"],axis=1,inplace=True)
TotalData.rename({"회사명_x":'회사명'},axis=1,inplace=True)
TotalData.head(2)

,회사명,영업이익20년1Q,영업이익20년2Q,영업이익20년3Q,순이익19년4Q,순이익20년1Q,순이익20년2Q,순이익20년3Q,시가총액(억),발표PSR,...,영업이익22년2Q,영업이익22년3Q,순이익20년4Q,순이익21년1Q,순이익21년2Q,순이익21년3Q,순이익21년4Q,순이익22년1Q,순이익22년2Q,순이익22년3Q
코드번호,,,,,,,,,,,,,,,,,,,,,
A005930,삼성전자,"64,473","81,463","123,532","52,280","48,896","54,890","92,668",3611718,1.17,...,140970,108520,64455,70928,94507,120572,106431,111291,109545,91439
A000660,SK하이닉스,"8,028","19,491","13,019","-1,255","6,329","12,689","10,839",595506,1.21,...,41926,16556,17694,9904,19845,33108,33165,19788,28715,11067


In [7]:
# Sanity Checks
# row size: CurrentData > YOYData, YOYData가 CurrentData에 포함됨
print(
    len(YOYData['코드번호'].isin(CurrentData['코드번호'])) == len(YOYData) # all YOY tickers contained in current data
)

# thus, followings are new tickers added (actual change in number of firms/tickers from 2020 --> 2021)
# OK because newly added ones does not meet Quant conditions that use YOY Data 
healthydiff = CurrentData.shape[0] - YOYData.shape[0]

# number of old tickers lost from merging, for some other reasons (May not be OK, except those delisted)
unhealthydiff = TotalData.shape[0] - CurrentData.shape[0] + healthydiff


healthydiff, unhealthydiff

True


(65, -28)

In [8]:
# filter columns with OP and NI

# 영업이익 데이터 df
OP = ['영업이익' in col for col in TotalData.columns]
OPdf = TotalData.iloc[:,OP]

# 순이익 데이터 df
NI = ['순이익' in col for col in TotalData.columns]
NIdf = TotalData.iloc[:,NI]

OPdf.info()
NIdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2259 entries, A005930 to A121890
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   영업이익20년1Q  2259 non-null   object
 1   영업이익20년2Q  2259 non-null   object
 2   영업이익20년3Q  2259 non-null   object
 3   영업이익20년4Q  2259 non-null   int64 
 4   영업이익21년1Q  2259 non-null   int64 
 5   영업이익21년2Q  2259 non-null   int64 
 6   영업이익21년3Q  2259 non-null   int64 
 7   영업이익21년4Q  2259 non-null   int64 
 8   영업이익22년1Q  2259 non-null   int64 
 9   영업이익22년2Q  2259 non-null   int64 
 10  영업이익22년3Q  2259 non-null   int64 
dtypes: int64(8), object(3)
memory usage: 211.8+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 2259 entries, A005930 to A121890
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   순이익19년4Q  2259 non-null   object
 1   순이익20년1Q  2259 non-null   object
 2   순이익20년2Q  2259 non-null   object
 3   순이익20년3Q  2259 non-null 

In [9]:
# NI and OP: Convert all object dtype to float in the DF
for col in range(len(OPdf.columns)):
    if OPdf.iloc[:,col].dtype == 'O':
        OPdf.iloc[:,col] = OPdf.iloc[:,col].str.replace(",","").astype(float)
    else:
        continue
        
for col in range(len(NIdf.columns)):
    if NIdf.iloc[:,col].dtype == 'O':
        NIdf.iloc[:,col] = NIdf.iloc[:,col].str.replace(",","").astype(float)
    else:
        continue

OPdf.info()
NIdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2259 entries, A005930 to A121890
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   영업이익20년1Q  2030 non-null   float64
 1   영업이익20년2Q  2030 non-null   float64
 2   영업이익20년3Q  2030 non-null   float64
 3   영업이익20년4Q  2259 non-null   int64  
 4   영업이익21년1Q  2259 non-null   int64  
 5   영업이익21년2Q  2259 non-null   int64  
 6   영업이익21년3Q  2259 non-null   int64  
 7   영업이익21년4Q  2259 non-null   int64  
 8   영업이익22년1Q  2259 non-null   int64  
 9   영업이익22년2Q  2259 non-null   int64  
 10  영업이익22년3Q  2259 non-null   int64  
dtypes: float64(3), int64(8)
memory usage: 211.8+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 2259 entries, A005930 to A121890
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   순이익19년4Q  2030 non-null   float64
 1   순이익20년1Q  2030 non-null   float64
 2   순이익20년2Q  2030 non-null   float64
 3   순이익20

In [10]:
# 4QTrailing 영업이익 & 순이익
FourQtrailingOP = OPdf.rolling(4,axis=1).sum().iloc[:,3:] # 8개
FourQtrailingNI = NIdf.rolling(4,axis=1).sum().iloc[:,3:] # 9개


# 영업이익 4QTrailing GEOMEAN
# 영업이익이 시간순일때 4QTrailing 영업이익 YoY gross return 계산 (= simple return + 1)
for i in range(4):
    new_col = f"YoY 4QTrailing OP Q{i+1}"
    FourQtrailingOP[new_col] = FourQtrailingOP.iloc[:,i+4] / FourQtrailingOP.iloc[:,i]

# 계산한 4QTrailing 영업이익 YoY gross return으로(최근 추가되어 오른쪽 4개 컬럼) GEOMEAN 계산 
FourQtrailingOP["4QTrailing OP GEOMEAN"] = gmean([
    FourQtrailingOP.iloc[:,-4],
    FourQtrailingOP.iloc[:,-3],
    FourQtrailingOP.iloc[:,-2],
    FourQtrailingOP.iloc[:,-1]
]) - 1


### FILTER CONDITIONS
# 4QTrailing 영업이익 GEOMEAN > 0.1
condition_OPGeomean = FourQtrailingOP["4QTrailing OP GEOMEAN"] > 0.1


# 4QTrailing 순이익
# Computing Periods settings: 9개 분기중 첫번째 제거 --> 8개 분기
FourQtrailingNI8 = NIdf.rolling(4,axis=1).sum().iloc[:,4:]

# 현 분기 YoY growth, simple return
YY,Q = naming(path)
FourQtrailingNI8["4QTrailing NI YoYGrowth"] = FourQtrailingNI8[f"순이익{YY}년{Q}"]/FourQtrailingNI8[f"순이익{YY-1}년{Q}"] -1


### FILTER CONDITION
# 4QTrailing 순이익 현 분기 YoY growth > 0
condition_4QTrailingNIyoygrowth = FourQtrailingNI8["4QTrailing NI YoYGrowth"] > 0

### FILTER CONDITION
# 4QTrailing 순이익 현 분기 YoY growth > 0, 9분기 all True
condition_9QTrailingNIpositive = (FourQtrailingNI > 0).all(axis=1);

In [11]:
# Computing Periods settings
YY,Q = naming(path) # YoY = YY,Q & YY-1,Q
YY0, Q0 = backwardQcounter(YY,Q,N=1) # QoQ = YY,Q & YY0,Q0(N=1)

# 영업이익 YoY, QoQ growth
OPdf["Q4 YoY OP growth"] = OPdf[f"영업이익{YY}년{Q}"]/OPdf[f"영업이익{YY-1}년{Q}"] - 1
OPdf["Q4 QoQ OP growth"] = OPdf[f"영업이익{YY}년{Q}"]/OPdf[f"영업이익{YY0}년{Q0}"] - 1

### FILTER CONDITION
# 영업이익 YoY, QoQ growth > 0
condition_OPYoYcurrentQgrowth = OPdf["Q4 YoY OP growth"] > 0
condition_OPQoQcurrentQgrowth = OPdf["Q4 QoQ OP growth"] > 0

# 순이익 YoY, QoQ growth
NIdf["Q4 YoY OP growth"] = NIdf[f"순이익{YY}년{Q}"]/NIdf[f"순이익{YY-1}년{Q}"] - 1
NIdf["Q4 QoQ OP growth"] = NIdf[f"순이익{YY}년{Q}"]/NIdf[f"순이익{YY0}년{Q0}"] - 1

### FILTER CONDITION
# 순이익 YoY, QoQ growth > 0
condition_NIYoYcurrentQgrowth = NIdf["Q4 YoY OP growth"] > 0
condition_NIQoQcurrentQgrowth = NIdf["Q4 QoQ OP growth"] > 0


### FILTER CONDITION
# Valuations conditions
condition_POR = (0 <= TotalData["발표POR"]) & (TotalData["발표POR"] <= 10)
condition_CFO = TotalData['영업활동현금흐름(억)'] > 0;

In [12]:
filteredDF = TotalData[condition_OPGeomean & 
   condition_4QTrailingNIyoygrowth & 
   condition_9QTrailingNIpositive & 
   condition_OPYoYcurrentQgrowth & 
   condition_OPQoQcurrentQgrowth & 
   condition_NIYoYcurrentQgrowth &
   condition_NIQoQcurrentQgrowth &
   condition_POR &
   condition_CFO]

# Valuation 지표 컬럼 추가
filteredDF["PCR"] = filteredDF['시가총액(억)'] / filteredDF['영업활동현금흐름(억)']
filteredDF["MV/EBITDA"] = filteredDF['시가총액(억)'] / filteredDF['EBITDA(억)']

filteredDF = filteredDF[filteredDF['PCR'] > 0]
filteredDF = filteredDF[filteredDF['발표PER'] > 0]
filteredDF = filteredDF[filteredDF['발표PBR'] > 0]
filteredDF = filteredDF[filteredDF['MV/EBITDA'] > 0]
filteredDF = filteredDF[filteredDF['발표PSR'] > 0]

filteredDF['PCR'] = filteredDF['PCR'].rank(ascending=True)
filteredDF['발표PER'] = filteredDF['발표PER'].rank(ascending=True)
filteredDF['발표PBR'] = filteredDF['발표PBR'].rank(ascending=True)
filteredDF['MV/EBITDA'] = filteredDF['MV/EBITDA'].rank(ascending=True)
filteredDF['발표PSR'] = filteredDF['발표PSR'].rank(ascending=True)

filteredDF["가치지표통합순위"] = np.mean(filteredDF[['발표PSR','발표PBR','발표PER',"PCR","MV/EBITDA"]],axis=1)


# 정렬기준 사용가능: "가치지표통합순위",'발표PSR','발표PBR','발표PER',"PCR","MV/EBITDA"
filtered_list = filteredDF["PCR"].sort_values().index.to_list() 
filteredDF.loc[filtered_list].head(5)

,회사명,영업이익20년1Q,영업이익20년2Q,영업이익20년3Q,순이익19년4Q,순이익20년1Q,순이익20년2Q,순이익20년3Q,시가총액(억),발표PSR,...,순이익21년1Q,순이익21년2Q,순이익21년3Q,순이익21년4Q,순이익22년1Q,순이익22년2Q,순이익22년3Q,PCR,MV/EBITDA,가치지표통합순위
코드번호,,,,,,,,,,,,,,,,,,,,,
A000540,흥국화재,-85,237,9,5,-63,198,10,2011,1.0,...,149,128,322,22,486,215,449,1.0,4.0,1.6
A036710,심텍홀딩스,132,297,308,12,37,107,93,1661,3.0,...,23,60,152,39,172,94,278,2.0,1.0,4.1
A138040,메리츠금융지주,"2,683","3,512","3,625","1,134",917,"1,445","1,305",48796,48.0,...,1787,1864,2234,2008,3352,2311,3205,3.0,8.0,22.8
A009970,영원무역홀딩스,572,596,"1,159",265,304,181,385,8100,6.0,...,373,382,704,750,681,1020,1395,4.0,2.0,3.6
A007810,코리아써키트,89,16,91,-61,81,36,73,3212,5.0,...,68,29,231,234,203,170,274,5.0,5.0,7.8


In [13]:
#YY,Q = naming(path)
Qclassify = {'06':'1Q','09':'2Q','12':'3Q','03':'4Q'}

MM = list(Qclassify.keys())[list(Qclassify.values()).index(Q)]

if Q == '4Q': # 현 분기를 날짜로 표현할때 4Q = 분기로 표현한 Year + 1
    YY += 1

current = f"현 분기: 20{YY} {MM}월, {Q}" 
print(current)

현 분기: 2022 12월, 3Q


In [14]:
### FnGuide 크롤링은 최근 4분기 for Quarterly, 최근 4년(이번 연도 진행중 포함) for Annual만 제공
# 최근 분기 영업활동현금흐름 크롤링 & 양수 확인
bool_list = []

for company in filtered_list:
    ### 크롤링할 영활현 Date설정 분기: "03"(4Q), "06"(1Q), "09"(2Q), "12"(3Q)
    # 가장 최근 분기 참고
    #YY,Q = naming(path)
    MM = list(Qclassify.keys())[list(Qclassify.values()).index(Q)]
    OCF = searchCF(company,Quarterly,"20"+str(YY),MM)
    
    try:
        if OCF == None:
            if Q == '4Q': # 현 날짜를 분기로 표현할때 4Q = 날짜로 표현한 Year - 1
                YY -= 1 
            MM = list(Qclassify.keys())[list(Qclassify.values()).index(Q)-1]
            OCF = searchCF(company,Quarterly,"20"+str(YY),MM)
            #print(company,OCF, OCF > 0, MM)
        if OCF > 0:
            bool_list.append(True)
        else: # including nan values for companies without quarterly OCF data
            bool_list.append(False)
    
    except TypeError as e:
        print(company, "None")
        bool_list.append(False)
        continue
     
    print(company,OCF, OCF > 0, YY,MM)

A000540 834.0 True 22 09
A036710 957.0 True 22 09
A138040 22857.0 True 22 09
A009970 2600.0 True 22 09
A007810 489.0 True 22 09
A010780 696.0 True 22 09
A007340 557.0 True 22 09
A010100 83.0 True 22 09
A000060 4364.0 True 22 09
A265520 294.0 True 22 09
A155660 215.0 True 22 09
A000990 2108.0 True 22 09
A009070 200.0 True 22 09
A066900 -18.0 False 22 09
A222800 1183.0 True 22 09
A008260 -10.0 False 22 09
A045100 220.0 True 22 09
A067280 167.0 True 22 09
A195870 602.0 True 22 09
A090460 811.0 True 22 09
A001340 51.0 True 22 09
A215200 415.0 True 22 09
A008370 49.0 True 22 09
A111770 3156.0 True 22 09
A183300 300.0 True 22 09
A011070 -324.0 False 22 09
A356860 163.0 True 22 09
A319660 175.0 True 22 09
A036670 69.0 True 22 09
A155650 95.0 True 22 09
A058630 66.0 True 22 09
A205100 38.0 True 22 09
A014570 36.0 True 22 09
A140070 213.0 True 22 09
A007570 102.0 True 22 09
A093320 95.0 True 22 09
A003220 125.0 True 22 09
A030000 1929.0 True 22 09
A083450 155.0 True 22 09
A094840 -1.0 False 22 

In [15]:
### FILTER CONDITION
# 영업활동현금흐름 양수 필터링
filtered_arr = np.array(filtered_list)
bool_arr = np.array(bool_list)
finalist = filtered_arr[bool_arr]

if Q == '4Q': # 현 분기를 날짜로 표현할때 4Q = 분기로 표현한 Year + 1
    YY += 1
print(current)
filteredDF.loc[finalist]

현 분기: 2022 12월, 3Q


,회사명,영업이익20년1Q,영업이익20년2Q,영업이익20년3Q,순이익19년4Q,순이익20년1Q,순이익20년2Q,순이익20년3Q,시가총액(억),발표PSR,...,순이익21년1Q,순이익21년2Q,순이익21년3Q,순이익21년4Q,순이익22년1Q,순이익22년2Q,순이익22년3Q,PCR,MV/EBITDA,가치지표통합순위
코드번호,,,,,,,,,,,,,,,,,,,,,
A000540,흥국화재,-85,237,9,5,-63,198,10,2011,1.0,...,149,128,322,22,486,215,449,1.0,4.0,1.6
A036710,심텍홀딩스,132,297,308,12,37,107,93,1661,3.0,...,23,60,152,39,172,94,278,2.0,1.0,4.1
A138040,메리츠금융지주,"2,683","3,512","3,625","1,134",917,"1,445","1,305",48796,48.0,...,1787,1864,2234,2008,3352,2311,3205,3.0,8.0,22.8
A009970,영원무역홀딩스,572,596,"1,159",265,304,181,385,8100,6.0,...,373,382,704,750,681,1020,1395,4.0,2.0,3.6
A007810,코리아써키트,89,16,91,-61,81,36,73,3212,5.0,...,68,29,231,234,203,170,274,5.0,5.0,7.8
A010780,아이에스동서,281,569,716,169,84,192,836,9206,12.0,...,119,-2,292,641,792,519,545,6.0,7.0,9.7
A007340,디티알오토모티브,203,27,258,134,204,10,166,6896,9.0,...,206,185,91,326,283,482,839,7.0,6.0,8.2
A010100,한국프랜지,63,-56,59,4,69,-53,55,877,2.0,...,90,37,21,38,70,76,195,8.0,3.0,3.6
A000060,메리츠화재,"1,463","1,462","1,545",675,"1,027","1,066","1,108",51657,14.0,...,1315,1640,1708,1936,2148,2323,2597,9.0,23.0,24.5


In [16]:
# filtered: 
len(filtered_arr) - len(finalist)

5

**END**